In [1]:
import shutil
import numpy as np
import matplotlib.pyplot as plt

from pyiron import Project
import pyiron_contrib
import pyiron
import pyiron_base
import h5py
import phonopy

### Check paths and versions

In [2]:
print(pyiron.__file__)
print(pyiron_base.__file__)
print(pyiron_contrib.__file__)
print(phonopy.__file__)

/u/caras/pyiron_repos/pyiron/pyiron/__init__.py
/u/caras/pyiron_repos/pyiron_base/pyiron_base/__init__.py
/u/caras/pyiron_repos/pyiron_contrib/pyiron_contrib/__init__.py
/u/caras/external_packages/phonopy/__init__.py


In [3]:
print(pyiron.__version__)
print(h5py.__version__)

0.3.17
3.1.0


In [4]:
pr = Project('Database1')
pr.remove_jobs(recursive=True)

# create version txt file
f = open(pr.path + "critical_versions.txt", "a")
f.write('# These are some important versions and SHAs used in this project.\n')
f.write("Pyiron version: {}\n".format(pyiron.__version__))
f.write("h5py version: {}\n".format(h5py.__version__))
f.write("phonopy file: {}\n".format(phonopy.__file__))
repo_df = pr.get_repository_status()
for i in repo_df.index.to_list():
    f.write(repo_df.loc[i,'Module']+'='+repo_df.loc[i,'Git head']+'\n')
f.close()

pr.get_repository_status() 

Are you sure you want to delete all jobs from 'Database1'? y/(n)y


,Module,Git head
0,pyiron_base,862be30490fda1b94b2e83788aa7c5ed45bf7d4d
1,pyiron,d410e585babd3b07326853de38134bd182b528bd
2,pyiron_contrib,783f52634780db0daac40b9bcba1d7ae2a06b683
3,pyiron_mpie,36862f9f1e51a13e818495f4354c4defb88d63ac


#### Target structures

In [5]:
ca_bcc = pr.create_ase_bulk(name='Ca', crystalstructure='bcc', a=4.38,
                                cubic=True).repeat(4)
ca_fcc = pr.create_ase_bulk(name='Ca', crystalstructure='fcc', a=5.576,
                                cubic=True).repeat(3)

sr_bcc = pr.create_ase_bulk(name='Sr', crystalstructure='bcc', a=4.741,
                                cubic=True).repeat(4)
sr_fcc = pr.create_ase_bulk(name='Sr', crystalstructure='fcc', a=6.00,
                                cubic=True).repeat(3)

ti_hcp = pr.create_ase_bulk(name='Ti', crystalstructure='hcp',
                                ).repeat(4)
ti_bcc = pr.create_ase_bulk(name='Ti', crystalstructure='bcc', a=3.25,
                                cubic=True).repeat(4)

zr_hcp = pr.create_ase_bulk(name='Zr', crystalstructure='hcp',
                                ).repeat(4)
zr_bcc = pr.create_ase_bulk(name='Zr', crystalstructure='bcc', a=3.584,
                                cubic=True).repeat(4)

y_hcp = pr.create_ase_bulk(name='Y', crystalstructure='hcp',
                           ).repeat(4)
y_bcc = pr.create_ase_bulk(name='Y', crystalstructure='bcc', a=4.026,
                                cubic=True).repeat(4)

al_bcc = pr.create_ase_bulk(name='Al', crystalstructure='bcc', a=3.22,
                                cubic=True).repeat(4)
al_fcc = pr.create_ase_bulk(name='Al', crystalstructure='fcc', a=4.039,
                                cubic=True).repeat(3)
al_hcp = pr.create_ase_bulk(name='Al', crystalstructure='hcp', a=2.87, c=4.66
                                ).repeat(4)

cu_bcc = pr.create_ase_bulk(name='Cu', crystalstructure='bcc', a=2.862,
                                cubic=True).repeat(4)
cu_fcc = pr.create_ase_bulk(name='Cu', crystalstructure='fcc', a=3.608,
                                cubic=True).repeat(3)
cu_hcp = pr.create_ase_bulk(name='Cu', crystalstructure='hcp', a=2.545, c=4.185
                                 ).repeat(4)
## Reorganize
structure_list = [ca_bcc, ca_fcc, sr_bcc, sr_fcc, ti_hcp, ti_bcc, zr_hcp, zr_bcc, y_hcp, y_bcc, al_bcc, al_fcc, al_hcp, cu_bcc, cu_fcc, cu_hcp]
structure_names = ['Ca_bcc', 'Ca_fcc', 'Sr_bcc', 'Sr_fcc', 'Ti_hcp', 'Ti_bcc', 'Zr_hcp', 'Zr_bcc', 'Y_hcp', 'Y_bcc', 'Al_bcc', 'Al_fcc', 'Al_hcp', 'Cu_bcc', 'Cu_fcc', 'Cu_hcp']
strain_list = np.around(np.linspace(0.9, 1.1, num=7),decimals=3)

#### Select potentials

In [6]:
al_pot = ['1999--Mishin-Y--Al--LAMMPS--ipr1']
ti_pot = ['1992--Ackland-G-J--Ti--LAMMPS--ipr1']
ca_pot = ['Morse_Shifted_GirifalcoWeizer_1959HighCutoff_Ca__MO_159753408472_003']
zr_pot = ['1995--Ackland-G-J--Zr--LAMMPS--ipr1']
y_pot =  ['2013--Ko-W-S--Y--LAMMPS--ipr1']
sr_pot = ['Morse_Shifted_GirifalcoWeizer_1959HighCutoff_Sr__MO_497591319122_003']
cu_pot = ['Morse_Shifted_GirifalcoWeizer_1959HighCutoff_Cu__MO_151002396060_003']
pot_dict={'Al':al_pot, 'Ti':ti_pot, 'Ca':ca_pot, 'Zr':zr_pot, 'Y':y_pot, 'Sr':sr_pot, 'Cu':cu_pot}

### Plot phonon bandstructures and harmonic free energy differences

In [7]:
def create_lammps_job(project, basis, potential, name, pressure = None):

    job_ref = project.create_job(job_type=pr.job_type.Lammps,
                                 job_name=name)
    job_ref.structure = basis
    job_ref.potential = potential
    
    job_ref.calc_minimize(ionic_energy_tolerance=1e-10,
                          ionic_force_tolerance=1e-12,
                          max_iter=10000,
                          pressure=pressure)
    
    return job_ref

def make_phonopy_job(project, basis, potential):
    pr_phon = project

    ref_job = pr_phon.create_job(pr.job_type.Lammps,'phononjob_ref')
    ref_job.structure = basis.copy()
    ref_job.potential = potential
    
    phonopy_job = pr_phon.create_job(pr.job_type.PhonopyJob,'phononjob')
    phonopy_job.ref_job = ref_job
    phonopy_job.input['primitive_matrix'] = 'auto'
    phonopy_job.server.cores = 4
    phonopy_job.server.queue = 'cmti'
    phonopy_job.server.run_time = 1000

    return phonopy_job

In [8]:
for structure_name,structure in zip(structure_names, structure_list):
    group_name = structure_name
    pr_s = pr.create_group(group_name)

    potential_list = pot_dict[''.join([i for i in structure.get_chemical_formula() if not i.isdigit()])]
    for potential in potential_list:
        pr_sp = pr_s.create_group(potential)

        for strain in strain_list:
            # sps stands for structure,potential,strain
            pr_sps = pr_sp.create_group(str(strain).replace('.','_'))
            print('Name: {}'.format(pr_sps.name))
            strain_structure = structure.copy()
            strain_structure.set_cell(cell=structure.cell*strain**(1/3),
                                      scale_atoms=True)
            energyjob = create_lammps_job(pr_sps, strain_structure,
                                          potential, 'energyjob', pressure = None)
            energyjob.run()
            phononjob = make_phonopy_job(pr_sps,
                                         strain_structure, potential)
            
            phononjob.run()

The job energyjob was saved and received the ID: 10761054
The job phononjob was saved and received the ID: 10761055
Queue system id:  1594632
The job energyjob was saved and received the ID: 10761056
The job phononjob was saved and received the ID: 10761057
Queue system id:  1594633
The job energyjob was saved and received the ID: 10761058
The job phononjob was saved and received the ID: 10761059
Queue system id:  1594634
The job energyjob was saved and received the ID: 10761060
The job phononjob was saved and received the ID: 10761061
Queue system id:  1594635
The job energyjob was saved and received the ID: 10761062
The job phononjob was saved and received the ID: 10761063
Queue system id:  1594636
The job energyjob was saved and received the ID: 10761064
The job phononjob was saved and received the ID: 10761067
Queue system id:  1594637
The job energyjob was saved and received the ID: 10761068
The job phononjob was saved and received the ID: 10761070
Queue system id:  1594638
The jo

The job energyjob was saved and received the ID: 10761224
The job phononjob was saved and received the ID: 10761226
Queue system id:  1594690
The job energyjob was saved and received the ID: 10761227
The job phononjob was saved and received the ID: 10761229
Queue system id:  1594691
The job energyjob was saved and received the ID: 10761230
The job phononjob was saved and received the ID: 10761233
Queue system id:  1594692
The job energyjob was saved and received the ID: 10761234
The job phononjob was saved and received the ID: 10761236
Queue system id:  1594693
The job energyjob was saved and received the ID: 10761237
The job phononjob was saved and received the ID: 10761238
Queue system id:  1594694
The job energyjob was saved and received the ID: 10761239
The job phononjob was saved and received the ID: 10761242
Queue system id:  1594695
The job energyjob was saved and received the ID: 10761244
The job phononjob was saved and received the ID: 10761245
Queue system id:  1594696
The jo